In [2]:
import os
import json
from collections import deque
from typing import Dict, List, Optional, Any, Union
from pydantic import BaseModel

from langchain.agents import ZeroShotAgent, Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain import OpenAI, SerpAPIWrapper, LLMChain, PromptTemplate
from langchain.chat_models import ChatOpenAI


from langchain.vectorstores import Chroma
from langchain.docstore import InMemoryDocstore

from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
import chromadb
import re


# Get human objective
# refine it via human in the loop and via adding context (short/long term memory search) -> the output is a json dict of the design
# build the modules and test benches usiiing short and long term memory

In [3]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY")

In [4]:
llm = ChatOpenAI(model='gpt-4', temperature=0)
bigllm = ChatOpenAI(model='gpt-4-1106-preview', temperature=0)

In [5]:
from FPGA_AGI.tools import *
from FPGA_AGI.agents import *
from FPGA_AGI.utils import *

## FPGA-AGI

In [8]:
from pydantic import BaseModel
import warnings

class FPGA_AGI(BaseModel):
    verbose: bool = False
    solution_num: int = 0
    requirement_agent_executor: RequirementAgentExecutor = None
    module_agent_executor: ModuleAgentExecutor = None
    hdl_agent_executor: HdlAgentExecutor = None
    requirement: str
    project_details: ProjectDetails
    module_list: List
    codes: List
    module_list_str: List

    def __init__(self, **data):
        super().__init__(**data)
        if (self.requirement_agent_executor is None) or (self.module_agent_executor is None) or (self.hdl_agent_executor is None):
            warnings.warn("RequirementAgentExecutor, ModuleAgentExecutor and HdlAgentExecutor are not set. Please use the from_llm class method to properly initialize it.", UserWarning)

    @classmethod
    def from_llm(cls, llm, verbose=False):
        tools = [
            web_search_tool,
            document_search_tool,
            human_input_tool
        ]
        requirement_agent_executor = RequirementAgentExecutor.from_llm_and_tools(llm=llm, tools=tools, verbose=verbose)
        tools = [
            web_search_tool,
            document_search_tool,
        ]
        module_agent_executor = ModuleAgentExecutor.from_llm_and_tools(llm=llm, tools=tools, verbose=verbose)
        tools = [
            #web_search_tool,
            document_search_tool,
        ]
        hdl_agent_executor = HdlAgentExecutor.from_llm_and_tools(llm=llm, tools=tools, verbose=verbose)
        return cls(verbose=verbose, requirement_agent_executor=requirement_agent_executor, module_agent_executor=module_agent_executor, hdl_agent_executor=hdl_agent_executor)

    def _full(self, objective):
        if (self.requirement_agent_executor is None) or (self.module_agent_executor is None) or (self.hdl_agent_executor is None):
            raise Exception("RequirementAgentExecutor, ModuleAgentExecutor and HdlAgentExecutor are not set. Please use the from_llm class method to properly initialize it.")
        self.requirement = self.requirement_agent_executor.run(objective)
        self.project_details = extract_project_details(self.requirement)
        self.module_list = self.module_agent_executor.run(Goals=self.project_details.goals, Requirements=self.project_details.requirements, Constraints=self.project_details.constraints)
        self.module_list = extract_json_from_string(self.module_list)
        self.codes = []
        self.module_list_str = [str(item) for item in self.module_list]
        #current_modules_verified = json.loads(current_modules_verified)
        for module in module_list:
            self.code = self.hdl_agent_executor.run(Goals=project_details.goals, Requirements=project_details.requirements, Constraints=project_details.constraints, module_list='\n'.join(module_list_str), codes='\n'.join(codes), module=str(module))
            self.codes.append(code)
        save_solution(self.codes, solution_num=self.solution_num)
        self.solution_num += 1

    def _module(self, objective):
        self.solution_num += 1

    def _hdl(self, objective):
        self.solution_num += 1

    def __call__(self, objective ,action_type='full'):
        if action_type == 'full':
            self._full(objective)
        elif action_type == 'module':
            self._module(objective)
        elif action_type == 'hdl':
            return self._hdl(objective)
        else:
            raise ValueError("Invalid action type specified.")


In [9]:
fpga_agi_instance = FPGA_AGI.from_llm(llm=bigllm, verbose=True)
fpga_agi_instance(objective="generate a 32 bit full adder in verilog" ,action_type='full')



> Entering new RequirementAgentExecutor chain...
Thought: The user has asked for a comprehensive explanation of the key functions, goals, and specific requirements necessary for a design project, but the final question seems to be a request for Verilog code for a 32-bit full adder, which is not aligned with the initial request. The user has explicitly stated that the job is not to write any code but to focus on goals, requirements, and constraints. Therefore, I will address the initial request and not the final question about generating Verilog code.

Action: I will start by defining the goals for an FPGA design project that might involve a 32-bit full adder as a component.

Final Answer:
- Goals:
  1. To design an FPGA-based system that can perform 32-bit addition operations with high speed and accuracy.
  2. To ensure the design is optimized for the chosen deployment platform, taking into account its specific resources and limitations.
  3. To create a design that is scalable and c

ValueError: "FPGA_AGI" object has no field "requirement"